In [12]:
import numpy as np
import pandas as pd
import scanpy as sc
import scrublet as scr
import seaborn as sns
import scipy.stats
import anndata
import os

In [4]:
adata = sc.read_h5ad("./final_global_annotated.h5ad")

In [5]:
def bh(pvalues):
    '''
    Computes the Benjamini-Hochberg FDR correction.

    Input:
        * pvals - vector of p-values to correct
    '''
    n = int(pvalues.shape[0])
    new_pvalues = np.empty(n)
    values = [ (pvalue, i) for i, pvalue in enumerate(pvalues) ]
    values.sort()
    values.reverse()
    new_values = []
    for i, vals in enumerate(values):
        rank = n - i
        pvalue, index = vals
        new_values.append((n/rank) * pvalue)
    for i in range(0, int(n)-1):
        if new_values[i] < new_values[i+1]:
            new_values[i+1] = new_values[i]
    for i, vals in enumerate(values):
        pvalue, index = vals
        new_pvalues[index] = new_values[i]
    return new_pvalues

In [6]:
### Change directory 
    
CWD = "/Users/jamrute/Desktop/"

from os import chdir
chdir(CWD)

In [11]:
adata

AnnData object with n_obs × n_vars = 144027 × 270
    obs: 'sample', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'propmt', 'rna_size', 'ngene', 'prot_size', 'bc', 'droplet_class', 'nCount_ADT', 'nFeature_ADT', 'nCount_CITE', 'nFeature_CITE', 'nCount_SCT', 'nFeature_SCT', 'SCT.weight', 'CITE.weight', 'sct.dsb_snn_res.0.1', 'sct.dsb_snn_res.0.2', 'sct.dsb_snn_res.0.3', 'sct.dsb_snn_res.0.4', 'sct.dsb_snn_res.0.5', 'sct.dsb_snn_res.0.6', 'sct.dsb_snn_res.0.7', 'sct.dsb_snn_res.0.8', 'seurat_clusters', 'Sample.name', 'Date.of.harvest', 'HF.etiology', 'Txp.LVAD', 'Race', 'Sex', 'Age', 'time.since.last.ischemic.event', 'BMI', 'HTN', 'DM', 'MI', 'Stent', 'CABG', 'CKD', 'Smoker', 'EF', 'CO', 'Intermacs..profile.1....profile.2...2..profile.3.3..profile.4.4..profile.5.5.', 'Atrial.arrh', 'Ventricular.arrh', 'MR', 'MS', 'AR', 'AS', 'Pacemaker', 'ICD', 'Temp.mech.support..pre.op.', 'Durable.mech.support..pre.op', 'GDMT', 'Inotropes', 'annotation.0.1', 'HF.etiology2'
    var: 'features'
    obsm: '

In [9]:
####### main
meta = adata.obs
sc.settings.verbosity = 1  # verbosity: errors (0), warnings (1), info (2), hints (3)

scorenames = ['scrublet_score','scrublet_cluster_score','bh_pval']
os.makedirs('scrublet-scores')

###

for sample in meta['sample'].unique():
    print(sample)
    #import data
    adata_sample = adata[adata.obs['sample'] == sample]
    
    #set up and run Scrublet
    scrub = scr.Scrublet(adata_sample.X)
    doublet_scores, predicted_doublets = scrub.scrub_doublets(verbose=False)
    scrub.plot_histogram();
    adata_sample.obs['scrublet_score'] = doublet_scores
    #overcluster prep. run turbo basic scanpy pipeline
    sc.pp.filter_genes(adata_sample, min_cells=3)
    sc.pp.normalize_per_cell(adata_sample, counts_per_cell_after=1e4)
    sc.pp.log1p(adata_sample)
    sc.pp.highly_variable_genes(adata_sample, min_mean=0.0125, max_mean=3, min_disp=0.5)
    adata_sample = adata_sample[:, adata_sample.var['highly_variable']]
    sc.pp.scale(adata_sample, max_value=10)
    sc.tl.pca(adata_sample, svd_solver='arpack')
    sc.pp.neighbors(adata_sample)
    #eoverclustering proper - do basic clustering first, then cluster each cluster
    sc.tl.louvain(adata_sample)
    for clus in np.unique(adata_sample.obs['louvain']):
        sc.tl.louvain(adata_sample, restrict_to=('louvain',[clus]))
        adata_sample.obs['louvain'] = adata_sample.obs['louvain_R']
    #compute the cluster scores - the median of Scrublet scores per overclustered cluster
    for clus in np.unique(adata_sample.obs['louvain']):
        adata_sample.obs.loc[adata_sample.obs['louvain']==clus, 'scrublet_cluster_score'] = \
            np.median(adata_sample.obs.loc[adata_sample.obs['louvain']==clus, 'scrublet_score'])
    #now compute doublet p-values. figure out the median and mad (from above-median values) for the distribution
    med = np.median(adata_sample.obs['scrublet_cluster_score'])
    mask = adata_sample.obs['scrublet_cluster_score']>med
    mad = np.median(adata_sample.obs['scrublet_cluster_score'][mask]-med)
    #let's do a one-sided test. the Bertie write-up does not address this but it makes sense
    pvals = 1-scipy.stats.norm.cdf(adata_sample.obs['scrublet_cluster_score'], loc=med, scale=1.4826*mad)
    adata_sample.obs['bh_pval'] = bh(pvals)
    #create results data frame for single sample and copy stuff over from the adata object
    scrublet_sample = pd.DataFrame(0, index=adata_sample.obs_names, columns=scorenames)
    for meta in scorenames:
        scrublet_sample[meta] = adata_sample.obs[meta]
    #write out complete sample scores
    scrublet_sample.to_csv('scrublet-scores/'+sample+'.csv')
    scrub.plot_histogram()

0


ValueError: n_components=30 must be between 1 and min(n_samples, n_features)=15 with svd_solver='arpack'

In [9]:
scrub1 = pd.read_csv("./scrublet-scores/1.csv", index_col=0)
scrub2 = pd.read_csv("./scrublet-scores/2.csv", index_col=0)
scrub3 = pd.read_csv("./scrublet-scores/3.csv", index_col=0)
scrub4 = pd.read_csv("./scrublet-scores/4.csv", index_col=0)
scrub5 = pd.read_csv("./scrublet-scores/5.csv", index_col=0)
scrub6 = pd.read_csv("./scrublet-scores/6.csv", index_col=0)
scrub7 = pd.read_csv("./scrublet-scores/7.csv", index_col=0)
scrub8 = pd.read_csv("./scrublet-scores/8.csv", index_col=0)
scrub9 = pd.read_csv("./scrublet-scores/9.csv", index_col=0)
scrub10 = pd.read_csv("./scrublet-scores/10.csv", index_col=0)

In [10]:
df_list = [scrub1, scrub2, scrub3, scrub4, scrub5, scrub6, scrub7, scrub8, scrub9, scrub10]
scrub_all = pd.concat(df_list)

In [11]:
scrub_all.to_csv('scrublet-scores/all.csv')